In [ ]:
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import matplotlib as mpl
from scipy import stats
from sklearn.linear_model import LinearRegression

from pathlib import Path
from SyringePumpAux import *


In [ ]:
#Path settings
figureTargetFolder = Path(r"C:\Users\bruno.cruz\Downloads\ExperimentalData\Figures")
dataFolder = Path(r"C:\Users\bruno.cruz\Downloads\ExperimentalData\BigVolumes")

#Global Viz settings
#Global Viz settings
sns.set_context('paper') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes
plt.rc("axes.spines", top=False, right=False)

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'

SeryngePallete = sns.color_palette("deep")

PumpID_colors = {"Pump_2" : SeryngePallete[2],
                "Pump_4" :  SeryngePallete[3]}

color_dict = {"Hamilton_5ml": SeryngePallete[0],
             "Hamilton_10ml": SeryngePallete[1]} 
             
Volume_colors = sns.color_palette("flare", 5)

Volume_colors_dict = {
    1000 : Volume_colors[0],
    2000 : Volume_colors[1],
    5000 : Volume_colors[2],
    10000 : Volume_colors[3]
}

default_fig_size = (6,4)
default_x_scale = (0 , 1.75)
default_marker_size = 10.

In [ ]:
csvFile = dataFolder / "BigVolumes_Final.csv"
# load raw data file
rawData = pd.read_csv(csvFile, sep = ",", skiprows=2)
rawData = rawData.rename(columns={"nsteps": "N_Steps", "weight": "Weight","Unnamed: 2": "PumpID"})
rawData["PumpID"] = rawData["PumpID"].replace({"pump2": "Pump_2", "pump4" : "Pump_4"})
rawData.head(2)


In [ ]:
SyringeRadius = 5.15
StepSizeMultiplier = 0.5
MotorFullStepRotation = 1.8 #degrees
ScrewPitch = 0.8 #millimeters per 360 degrees
fullStepStroke =  (MotorFullStepRotation/360.0) * ScrewPitch # millimeters
singleStepVolume = (np.pi * ((SyringeRadius) ** 2) ) * fullStepStroke * StepSizeMultiplier
print(singleStepVolume)
rawData["ExpectedVolume"] = rawData["N_Steps"] * singleStepVolume*1e-3


In [ ]:

force_0_intercept = True

for pumpID, pumpGroup in rawData.groupby("PumpID"):
    fig = plt.figure(figsize= default_fig_size)
    ExpectedVolume = pumpGroup["ExpectedVolume"].values
    MeasuredWeight = pumpGroup["Weight"].values

    regModel = LinearRegression(
        fit_intercept = not(force_0_intercept)).fit(ExpectedVolume.reshape(-1, 1), MeasuredWeight.reshape(-1, 1))

    xx = np.array((0,2)).reshape(-1,1)
    yy = regModel.predict(xx)
    if not(force_0_intercept):
        plt.plot(xx, yy, color = PumpID_colors[pumpID], alpha = 1, lw = 2,
        label = pumpID +r", $R^2=$"+
            str(round(regModel.score(ExpectedVolume.reshape(-1, 1), MeasuredWeight.reshape(-1, 1)),4))
            + r", $\beta_1=$" + str(np.round(regModel.coef_[0,0],4))
            + r", $\beta_0=$" + str(np.round(regModel.intercept_[0],4))
            )
    else:
        plt.plot(xx, yy, color = PumpID_colors[pumpID], alpha = 1, lw = 2,
        label = pumpID +r", $R^2=$"+
            str(round(regModel.score(ExpectedVolume.reshape(-1, 1), MeasuredWeight.reshape(-1, 1)),4))
            + r", $\beta_1=$" + str(np.round(regModel.coef_[0],4))
            )
        print(pumpID +r", $R^2=$"+
            str(round(regModel.score(ExpectedVolume.reshape(-1, 1), MeasuredWeight.reshape(-1, 1)),4))
            + r", $\beta_1=$" + str(np.round(regModel.coef_[0],4))
            )

    for N_Steps, N_stepsGroup in pumpGroup.groupby("N_Steps"):
        allVolumes = N_stepsGroup["Weight"].values
        # plt.scatter(N_stepsGroup["ExpectedVolume"].values, allVolumes,
        #             color = Volume_colors_dict[N_Steps],
        #             s = default_marker_size**2
        #         )
        plt.errorbar(N_stepsGroup["ExpectedVolume"].values[0], np.average(allVolumes), np.std(allVolumes),
                    color = Volume_colors_dict[N_Steps],
                    fmt='o', capsize=0, elinewidth = 3, ecolor = Volume_colors_dict[N_Steps],
                    mfc = Volume_colors_dict[N_Steps],
                    markeredgecolor = Volume_colors_dict[N_Steps], markersize = default_marker_size,
                    label = pumpID + "_" + str(N_Steps))

    plt.title("Big volumes_" + pumpID)
    plt.legend()
    plt.xlabel('ExpectedVolume (ml)')
    plt.ylabel('Mass [g]')
    plt.xlim(default_x_scale)
    plt.ylim(default_x_scale)
    saveFigure(fig, rootSaveFolder = figureTargetFolder)
    plt.show()

In [ ]:
force_0_intercept = True
fig = plt.figure(figsize= default_fig_size)
for pumpID, pumpGroup in rawData.groupby("PumpID"):
    ExpectedVolume = pumpGroup["ExpectedVolume"].values
    MeasuredWeight = pumpGroup["Weight"].values

    for N_Steps, N_stepsGroup in pumpGroup.groupby("N_Steps"):
        allVolumes = N_stepsGroup["Weight"].values

        plt.scatter(N_stepsGroup["ExpectedVolume"].values[0], 100.0*np.std(allVolumes)/np.average(allVolumes),
                    color = Volume_colors_dict[N_Steps], edgecolors = PumpID_colors[pumpID], s = default_marker_size**2,
                    label = pumpID + "_" + str(N_Steps))
plt.title('CV')
#plt.legend()
plt.xlabel('ExpectedVolume (ml)')
plt.ylabel('Coefficient of variation (%)')
plt.ylim([0, 1])
plt.xlim(default_x_scale)
saveFigure(fig, rootSaveFolder = figureTargetFolder)
plt.show()